In [20]:
import numpy as np
import pandas as pd
import seaborn as sns
from random import randint
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [21]:
def get_speed(mark):
#     "VW", "Honda", "Hyundai", "Skoda", "Vaz"
    if mark == "VW":
        sp = randint(80,140)
    elif mark == "Honda":
        sp =randint(80,130)
    elif mark == "Hyundai":
        sp = randint(80,120)
    elif mark == "Skoda":
        sp = randint(80,110)
    elif mark == "Vaz":
        sp = randint(80,100)    
    return sp

In [22]:
# Создаём сэмпл
n_samples = 1000
REPLACE_WITH = "n/a"
REPLACE_COUNT = int(n_samples/10)

driving_exp = np.random.choice(20, n_samples)
travel_time = np.random.choice(np.arange(3,6,0.2), n_samples)
stop_time =np.random.choice(range(5,30), n_samples)/60
stop_time = np.asarray(list(map(lambda x: round(x,2), stop_time)))
car_mark = np.random.choice(["VW", "Honda", "Hyundai", "Skoda", "Vaz"], n_samples)
breakage = np.random.choice(2, n_samples)
fix_time = list(map(lambda x: round(randint(5,120)/60,2) if x!=0 else 0,breakage))
average_speed =  [get_speed(mark) for mark in car_mark]
distance = map(int,(average_speed*travel_time - average_speed*stop_time - average_speed*breakage*fix_time) *1.5 + 22)

data_init = pd.DataFrame({'driving_exp':driving_exp,'travel_time': travel_time, 'stop_time': stop_time, 'car_mark':car_mark, 
                     'breakage': breakage, 'fix_time': fix_time, "average_speed":average_speed, "distance":distance })


# add missing values
data_init.breakage.iloc[np.random.choice(n_samples-1, REPLACE_COUNT, replace=False)] = None
data_init.driving_exp.iloc[np.random.choice(n_samples-1, 800, replace=False)] = None
data_init.average_speed.iloc[np.random.choice(n_samples-1, REPLACE_COUNT, replace=False)] = None
data_init.stop_time.iloc[np.random.choice(n_samples-1, REPLACE_COUNT, replace=False)] = None
data_init.head(20)


C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,driving_exp,travel_time,stop_time,car_mark,breakage,fix_time,average_speed,distance
0,NaN,4.2,0.15,Vaz,0.0,0.00,99.0,623
1,NaN,4.4,0.10,Honda,1.0,0.82,NaN,497
2,1.0,5.8,0.37,VW,0.0,0.00,117.0,974
3,NaN,3.2,0.43,Skoda,0.0,0.00,110.0,479
4,NaN,4.8,0.27,Skoda,1.0,0.40,106.0,678
5,NaN,4.8,0.35,Honda,0.0,0.00,107.0,736
6,NaN,4.8,0.23,Vaz,1.0,1.52,99.0,474
7,NaN,4.2,0.48,Honda,NaN,1.20,NaN,347
8,NaN,5.0,0.40,VW,0.0,0.00,80.0,574
9,7.0,3.2,0.47,Skoda,1.0,1.27,107.0,256


In [5]:
data = data_init.copy()
data


,driving_exp,travel_time,stop_time,car_mark,breakage,fix_time,average_speed,distance
0,NaN,4.0,0.15,Skoda,0.0,0.00,109.0,651
1,NaN,5.0,0.18,Honda,0.0,0.00,90.0,672
2,NaN,4.6,0.35,Honda,1.0,1.48,113.0,491
3,NaN,4.8,0.37,VW,0.0,0.00,98.0,673
4,NaN,4.6,NaN,VW,1.0,1.63,111.0,444
...,...,...,...,...,...,...,...,...
995,NaN,4.4,0.35,Vaz,1.0,0.73,99.0,515
996,NaN,5.4,0.35,Honda,NaN,0.52,111.0,776
997,4.0,3.6,0.43,Vaz,1.0,0.20,89.0,418
998,NaN,4.2,NaN,Skoda,0.0,0.00,86.0,540


 driving_exp  слишком много пропусков заменю индикатором

In [6]:
data['driving_exp_ind'] = data['driving_exp'].apply(lambda x: 1 if np.isnan(x) else 0)
data = data.drop(columns=['driving_exp'])

Заменяю пропуски в breakage модай

In [7]:
print(data['breakage'].mode()[0])
data.breakage.fillna(data['breakage'].mode()[0], inplace=True)
data

0.0


,travel_time,stop_time,car_mark,breakage,fix_time,average_speed,distance,driving_exp_ind
0,4.0,0.15,Skoda,0.0,0.00,109.0,651,1
1,5.0,0.18,Honda,0.0,0.00,90.0,672,1
2,4.6,0.35,Honda,1.0,1.48,113.0,491,1
3,4.8,0.37,VW,0.0,0.00,98.0,673,1
4,4.6,NaN,VW,1.0,1.63,111.0,444,1
...,...,...,...,...,...,...,...,...
995,4.4,0.35,Vaz,1.0,0.73,99.0,515,1
996,5.4,0.35,Honda,0.0,0.52,111.0,776,1
997,3.6,0.43,Vaz,1.0,0.20,89.0,418,0
998,4.2,NaN,Skoda,0.0,0.00,86.0,540,1


замена пропусков в stop_time

In [8]:
data.stop_time.mean(), data.stop_time.std(), data.stop_time.median()

(0.27876666666666683, 0.11935743330613423, 0.28)

In [9]:
data.stop_time.fillna(data['stop_time'].median(), inplace=True)
data.stop_time.mean(), data.stop_time.std()
# статистики изменились незначительно

(0.27889000000000025, 0.11322671146062757)

In [10]:
#one hot encoding for car_mark
dummies = pd.get_dummies(data.car_mark)
cols = dummies.columns
for col in cols:
    data[col] = dummies[col]
data = data.drop(columns=['car_mark'])
data

,travel_time,stop_time,breakage,fix_time,average_speed,distance,driving_exp_ind,Honda,Hyundai,Skoda,VW,Vaz
0,4.0,0.15,0.0,0.00,109.0,651,1,0,0,1,0,0
1,5.0,0.18,0.0,0.00,90.0,672,1,1,0,0,0,0
2,4.6,0.35,1.0,1.48,113.0,491,1,1,0,0,0,0
3,4.8,0.37,0.0,0.00,98.0,673,1,0,0,0,1,0
4,4.6,0.28,1.0,1.63,111.0,444,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,4.4,0.35,1.0,0.73,99.0,515,1,0,0,0,0,1
996,5.4,0.35,0.0,0.52,111.0,776,1,1,0,0,0,0
997,3.6,0.43,1.0,0.20,89.0,418,0,0,0,0,0,1
998,4.2,0.28,0.0,0.00,86.0,540,1,0,0,1,0,0


замена пропусков в average_speed на основании модели

In [11]:
cols = ["Honda", "Hyundai", "Skoda","VW", "Vaz"]
Xtmp = data[cols]
Xtmp["average_speed"] = data["average_speed"]
Xtmp = Xtmp.dropna() 
Ytmp = Xtmp["average_speed"]
Xtmp = Xtmp[cols]
model = LinearRegression().fit(Xtmp, Ytmp)
data.loc[data[data["average_speed"].isna()].index,["average_speed"]]=model.predict(data[data["average_speed"].isna()][cols])
data



<ipython-input-11-acc2183dfa3d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Xtmp["average_speed"] = data["average_speed"]


,travel_time,stop_time,breakage,fix_time,average_speed,distance,driving_exp_ind,Honda,Hyundai,Skoda,VW,Vaz
0,4.0,0.15,0.0,0.00,109.0,651,1,0,0,1,0,0
1,5.0,0.18,0.0,0.00,90.0,672,1,1,0,0,0,0
2,4.6,0.35,1.0,1.48,113.0,491,1,1,0,0,0,0
3,4.8,0.37,0.0,0.00,98.0,673,1,0,0,0,1,0
4,4.6,0.28,1.0,1.63,111.0,444,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
995,4.4,0.35,1.0,0.73,99.0,515,1,0,0,0,0,1
996,5.4,0.35,0.0,0.52,111.0,776,1,1,0,0,0,0
997,3.6,0.43,1.0,0.20,89.0,418,0,0,0,0,0,1
998,4.2,0.28,0.0,0.00,86.0,540,1,0,0,1,0,0


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   travel_time      1000 non-null   float64
 1   stop_time        1000 non-null   float64
 2   breakage         1000 non-null   float64
 3   fix_time         1000 non-null   float64
 4   average_speed    1000 non-null   float64
 5   distance         1000 non-null   int64  
 6   driving_exp_ind  1000 non-null   int64  
 7   Honda            1000 non-null   uint8  
 8   Hyundai          1000 non-null   uint8  
 9   Skoda            1000 non-null   uint8  
 10  VW               1000 non-null   uint8  
 11  Vaz              1000 non-null   uint8  
dtypes: float64(5), int64(2), uint8(5)
memory usage: 59.7 KB


In [15]:
y = data['distance']
X = data[["travel_time","stop_time", "breakage", "fix_time", "average_speed", "driving_exp_ind", "Honda", "Hyundai", "Skoda",
         "Vaz", "VW"]]
# print(X)
# print(y)
model = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42 )  
model.fit(X_train, y_train)
y_pred = model.predict( X_test )
r2_score(y_pred, y_test)
# модель неплохо обучает


0.9635233333349821

дабавлю признак = average_speed х travel_time

In [17]:
data["time_speed"] = data["travel_time"] * data["average_speed"]

повторю обучение

In [19]:
y = data['distance']
X = data[["travel_time","stop_time", "breakage", "fix_time", "average_speed", "driving_exp_ind", "Honda", "Hyundai", "Skoda",
         "Vaz", "VW", "time_speed"]]
# print(X)
# print(y)
model = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42 )  
model.fit(X_train, y_train)
y_pred = model.predict( X_test )
r2_score(y_pred, y_test)
# модель стала обучать немного лучше

0.9736530545876094